In [2]:
from z3 import *
from itertools import combinations as comb

In [38]:
data={
        "error":error[210:],
        "var":{"func":[
            {
                "state":"input",
                "type":"int",
                "name":"a",
                "coord":2,
                "value":value[210:],
                "ret":[]
            }
        ]}
    
}

In [40]:
synthesis().synthesis(data)

{'func': {'op': '1',
  'opr': [{'state': 'input',
    'type': 'int',
    'name': 'a',
    'coord': 2,
    'ret': []}]}}

In [23]:
data={'error': [0, 1],
  'var': {'func': [{'state': 'input',
     'type': 'int',
     'name': 'a',
     'coord': 'dep__src.c:5:14',
     'value': [0, 5],
     'ret': [{'coord': 'dep__src.c:10:9'},
      {'coord': 'dep__src.c:12:9'},
      {'coord': 'dep__src.c:12:16'}]}]}}

In [4]:
class synthesis:
    def __init__(self):
        pass    
    def tautology(self,a):
        return True
    '''
    def var(self,a):  
        return a!=0
    def not_var(self,a):
        return a==0
    '''
    def equ(self,a,c):
        return a==c
    def neq(self,a,c):
        return a!=c
    def leq(self,a,c):
        return a<=c
    def lss(self,a,c):
        return a<c
    def geq(self,a,c):
        return a>=c
    def gtr(self,a,c):    
        return a>c

        
    def unary_op(self,error,var):
        op={
            "1":self.tautology
        }
        res={}
        for name,spec in op.items():
            for ele in var:
                if len(ele["value"])!=len(error):
                    continue
                s=Solver()
                for i in range(len(error)):
                    if error[i]:
                        s.add(spec(ele["value"][i]))
                    else:
                        s.add(Not(spec(ele["value"][i])))
                sat=s.check()
                if sat==z3.z3.sat:
                    res["op"]=name
                    res["opr"]=[]
                    res["opr"].append({
                        "state":ele["state"],
                        "type":ele["type"],
                        "name":ele["name"],
                        "coord":ele["coord"],
                        "ret":ele["ret"]
                    })
                    return sat,res
        return z3.z3.unsat,res
                
    def binary_op(self,error,var):
        op={
            "==":self.equ,
            "!=":self.neq,
            "<=":self.leq,
            "<" :self.lss,
            ">=":self.geq,
            ">" :self.gtr,
        }
        res={}
        #var op constant
        for name,spec in op.items():
            for ele in var:
                if len(ele["value"])!=len(error):
                    continue
                s=Solver()
                cons=Int('constant')
                for i in range(len(error)):
                    if error[i]:
                        s.add(spec(ele["value"][i],cons))
                    else:
                        s.add(Not(spec(ele["value"][i],cons)))
                sat=s.check()
                if sat==z3.z3.sat:
                    res["op"]=name
                    res["opr"]=[]
                    res["opr"].append({
                        "state":ele["state"],
                        "type":ele["type"],
                        "name":ele["name"],
                        "coord":ele["coord"],
                        "ret":ele["ret"]
                    })
                    res["opr"].append({
                        "state":"element",
                        "type":"int",
                        "name":"_const",
                        "coord":None,
                        "const":s.model()[cons],
                        "ret":ele["ret"]
                    })
                    return sat,res
        #var op var
        for name,spec in op.items():
            var_comb=comb(var,2)
            for ele1,ele2 in var_comb:
                if len(ele1["value"])!=len(error) or len(ele2["value"])!=len(error):
                    continue
                s=Solver()
                for i in range(len(error)):
                    if error[i]:
                        s.add(spec(ele1["value"][i],ele2["value"][i]))
                    else:
                        s.add(Not(spec(ele1["value"][i],ele2["value"][i])))
                sat=s.check()
                if sat==z3.z3.sat:
                    res["op"]=name
                    res["opr"]=[]
                    res["opr"].append({
                        "state":ele1["state"],
                        "type":ele1["type"],
                        "name":ele1["name"],
                        "coord":ele1["coord"],
                        "ret":ele1["ret"]
                    })
                    res["opr"].append({
                        "state":ele2["state"],
                        "type":ele2["type"],
                        "name":ele2["name"],
                        "coord":ele2["coord"],
                        "ret":ele2["ret"]
                    })
                    return sat,res
            
            
        return z3.z3.unsat,res

    def triple_op(self,a,b,c):
        pass
    
    def synthesis(self,data):
        error=data["error"]
        out={}
        for func,var in data["var"].items():
            out[func]=None
            #unary operator
            sat,res=self.unary_op(error,var)
            if sat==z3.z3.sat:
                out[func]=res
                continue
            #binary operator
            sat,res=self.binary_op(error,var)
            if sat==z3.z3.sat:
                out[func]=res
                continue
        return out

In [5]:
synthesis().synthesis(data)

{'func': None}

In [45]:
s=Solver()

l_inhibit=0
l_up=1
l_down=2
l_c=3
l_x,l_y=Ints('l_x l_y')

inhibit=[1,1,1]
up=[0,11,-20]
down=[100,110,60]
c=Int('c')
x=[]
y=[]
r=[]
for i in range(3):
    x.append(If(l_x==l_inhibit,inhibit[i],If(l_x==l_up,up[i],If(l_x==l_down,down[i],c))))
    y.append(If(l_y==l_inhibit,inhibit[i],If(l_y==l_up,up[i],If(l_y==l_down,down[i],c))))
    r.append(x[i]+y[i])

s.add(l_x>=0,l_x<4,l_y>=0,l_y<4)
s.add(r[0]<=100,r[1]>110,r[2]>60)
print(s.check())
print(s.model())

sat
[c = 100, l_x = 3, l_y = 1]


In [50]:
s.model()[c]

100

In [29]:
s=Solver()
x=[4,5,6,8]
c=Int('c')
s.add(x[0]<c,x[1]<c,Not(x[2]<c),Not(x[3]<c))
print(s.check())
print(s.model())

sat
[c = 6]


In [5]:
s=Solver()
x=[4,5,6,8]
c=Int('c')
s.add(x[0]==c,x[1]==c,Not(x[2]==c),Not(x[3]==c))
print(s.check())

unsat


In [6]:
type(s.check())

z3.z3.CheckSatResult

In [9]:
s.check()==z3.z3.unsat

True

In [60]:
s=Solver()
c=Int("c")
s.add(True,True,False)

In [61]:
s.check()

unsat

In [58]:
s.model()

[c = 3]

In [30]:
z3.z3.unsat

unsat